# Carbon dioxide solubility in the NaCl-brine

<p class="acknowledgement">Written by Svetlana Kyas (ETH Zurich) on Feb 8th, 2022</p>

This tutorial demonstrates how to simulate the solubility of CO<sub>2</sub> gas in the NaCl-brine and its dependence on
the salinity of the brine (also referred to as a salting-out effect).

We start the definition of function `solubility_co2()` that calculates the solubility of CO<sub>2</sub> in the NaCl-brine
by returning the concentration of carbon that got dissolved in the brine.

In [ ]:
from reaktoro import *
import numpy as np

def solubility_co2(system, solver, T, P, mNaCl):

    # Initial amount of the CO2 gas
    n0CO2g = 10.0

    # Define initial chemical state corresponding to the NaCl-brine of the given concentration
    state = ChemicalState(system)
    state.setTemperature(T, "celsius")
    state.setPressure(P, "bar")
    state.set("H2O"   , 1.0   , "kg")
    state.set("CO2(g)", n0CO2g, "mol")
    state.set("Na+"   , mNaCl , "mol")
    state.set("Cl-"   , mNaCl , "mol")

    # Calculate equilibrium state
    res = solver.solve(state)

    # Throw exception if the equilibrium couldn't be found
    if not res.optima.succeeded:
        raise RuntimeError("Equilibrium calculation did not succeed!")

    # Fetch resulting aqueous properties of the chemical state
    aqprops = AqueousProps(state)

    # Return concentration of the carbon in the aqueous phase
    return aqprops.elementMolality("C")[0]

The chemical system and the corresponding equilibrium solver are initialized below.

In [ ]:
# Initialize a thermodynamic database
db = PhreeqcDatabase("phreeqc.dat")

# Create an aqueous phase automatically selecting all species with provided elements
aqueousphase = AqueousPhase(speciate("H O C Na Cl"))
aqueousphase.setActivityModel(chain(
    ActivityModelHKF(),
    ActivityModelDrummond("CO2"),
))

# Create a gaseous phase
gaseousphase = GaseousPhase("CO2(g)")
gaseousphase.setActivityModel(ActivityModelPengRobinson())

phases = Phases(db)
phases.add(aqueousphase)
phases.add(gaseousphase)

# Create the chemical system
system = ChemicalSystem(phases)

# Create the equilibrium solver
solver = EquilibriumSolver(system)

Finally, we define the range of temperatures and pressure for the equilibrium calculations and calculate CO<sub>2</sub>
solubilities for the range of the temperatures and different brine concentrations.

In [ ]:
T = np.arange(25.0, 90.0, 5.0)
P = 100.0

mCO2_nacl1 = [solubility_co2(system, solver, x, P, mNaCl=1.0) for x in T]
mCO2_nacl2 = [solubility_co2(system, solver, x, P, mNaCl=2.0) for x in T]
mCO2_nacl4 = [solubility_co2(system, solver, x, P, mNaCl=4.0) for x in T]

# Output the results
print(" ----------------------------------------------------------------")
print("  CO2 solubilities w.r.t. temperatures")
print(" ----------------------------------------------------------------")
print("   T    1 mol NaCl-brine   2 mol NaCl-brine   4 mol NaCl-brine")
for i in range(len(T)):
    print(f"{T[i]:4.0f}  {mCO2_nacl1[i]:18.4f} {mCO2_nacl2[i]:18.4f} {mCO2_nacl4[i]:18.4f}")

Below, we plot the solubility of CO<sub>2</sub>(g) as a function of pressure for different salinities of NaCl-brine.
As expected, the CO<sub>2</sub>(g)-solubility decreases with the increase of the temperature. Moreover, we see that
the solubility of the gas drops as the brine becomes more saline.

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
ax.plot(T, mCO2_nacl1, label=f"1 NaCl molal")
ax.plot(T, mCO2_nacl2, label=f"2 NaCl molal")
ax.plot(T, mCO2_nacl4, label=f"4 NaCl molal")
ax.legend(loc="best")
ax.grid(True)
ax.set(xlabel=r'Temperatures [$^\circ$]')
ax.set(ylabel='Solubility [mol/kgw]')
ax.set(title='Solubility of CO2 in NaCl brine, P = ' + str(P) + ' bar');